In [1]:
#libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import os
from IPython.display import Image

In [2]:
#settings
pd.set_option('display.max_column', None)
import warnings 
warnings.filterwarnings('ignore')

In [3]:
url='https://www.billboard.com/charts/hot-100/'

In [4]:
# 200 status code? Yes
response = requests.get(url)
response.status_code 

200

In [5]:
html = response.text
soup = BeautifulSoup(html, 'html.parser')
#soup()

In [6]:
#print(soup.prettify())

In [7]:
title = soup.title.string
title

'Billboard Hot 100™'

In [26]:
#First try to extract the name of the song returned the top row
name=[]
for element in soup.find_all("ul", class_="o-chart-results-list-row // lrv-a-unstyle-list lrv-u-flex u-height-200 u-height-100@mobile-max u-height-100@tablet-only lrv-u-background-color-white a-chart-has-chart-detail"):
    name.append(element.li.find_next("h3").get_text(strip=True))
name

['A Bar Song (Tipsy)']

In [10]:
# second try in extacting name of song and name of artist returned name of song and artist togther with excessive numerical data
text=[]
for element in soup.find_all("div",class_="o-chart-results-list-row-container"):
    text.append(element.ul.find_next('ul').get_text(strip=True))
text[0:2]

['A Bar Song (Tipsy)Shaboozey11241124',
 'I Had Some HelpPost Malone Featuring Morgan Wallen21202120']

In [11]:
song_list = []

for element in soup.find_all("div", class_="o-chart-results-list-row-container"):
    song_text = element.ul.find_next('ul').get_text(strip=True)
    cleaned_text = ''.join([i for i in song_text if not i.isdigit()])  # to removes digits at the end of the string
    song_list.append(cleaned_text)

song_list[0:2]   

['A Bar Song (Tipsy)Shaboozey',
 'I Had Some HelpPost Malone Featuring Morgan Wallen']

In [16]:
import re
song_dict = {}

for song in song_list:
    # to separate the song name and artist name use a regular expression to find the first place lowercase become uppercase
    match = re.search(r'([a-z\)])([A-Z])', song)
    
    if match: #to split song name and artist name
        split_index = match.end(1)  # position where the artist starts
        song_title = song[:split_index].strip()  # extract song title up to the transition
        artist = song[split_index:].strip()  # artist after the transition
        song_dict[song_title] = artist
#song_dict

In [18]:
df1 = pd.DataFrame(list(song_dict.items()), columns=['Song Title', 'Artist'])
df1

,Song Title,Artist
0,A Bar Song (Tipsy),Shaboozey
1,I Had Some Help,Post Malone Featuring Morgan Wallen
2,Espresso,Sabrina Carpenter
3,Die With A Smile,Lady Gaga & Bruno Mars
4,Birds Of A Feather,Billie Eilish
...,...,...
87,Diet Pepsi,Addison Rae
88,Coincidence,Sabrina Carpenter
89,Passport Junkie,Rod Wave
90,Circadian Rhythm,Drake


In [ ]:
# Another solution shared in class using attribute
song_titles=[]
for element in soup.find_all(attrs={'class':'o-chart-results-list-row-container'}):
    song_titles.append(element.h3.get_text(strip=True))

In [ ]:
#shared in class
band_name=[]
for element in soup.find_all(attrs={'class':'o-chart-results-list-row-container'}):
    band_name.append(element.h3.find_next('span').get_text(strip=True))